In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests, time, json, pytz, asyncio, random
import pandas as pd
import SpaceTraders as ST
from SpaceTraders import io, F_utils, F_nav, F_trade, F_extract, scripts, fleet_resource_manager
from SpaceTraders.controllers import system_market_intel as MI
from SpaceTraders.controllers import system_siphoners as SIPHON
from datetime import datetime

In [3]:
ST.get_request('/my/agent').json()

{'data': {'accountId': 'cmhhh9gk1002vtm16u5rl9pf0',
  'symbol': 'RYVIOS',
  'headquarters': 'X1-QT34-A1',
  'credits': 1173649,
  'startingFaction': 'COSMIC',
  'shipCount': 4}}

In [42]:
ST.get_request('/agents/HIVE').json()

{'data': {'symbol': 'HIVE',
  'headquarters': 'X1-QT34-A1',
  'credits': 30505752,
  'startingFaction': 'COSMIC',
  'shipCount': 17}}

In [5]:
F_nav.refuel_ship('RYVIOS-1')
F_nav.navigate_in_system('RYVIOS-1', 'X1-TP30-D41', 'BURN', verbose=True)

[INFO] Ship RYVIOS-1 refueled 0 units. Total cost: 0 cr (72 cr/u).
[ERROR] Ship RYVIOS-1 is already at destination X1-TP30-D41. Navigation cancelled.


False

In [9]:
F_nav.get_ship_nav('RYVIOS-1')

{'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-D41',
 'route': {'destination': {'symbol': 'X1-TP30-D41',
   'type': 'PLANET',
   'systemSymbol': 'X1-TP30',
   'x': 41,
   'y': -75},
  'origin': {'symbol': 'X1-TP30-H52',
   'type': 'MOON',
   'systemSymbol': 'X1-TP30',
   'x': 40,
   'y': -19},
  'departureTime': '2025-11-04T17:42:20.961Z',
  'arrival': '2025-11-04T17:42:55.961Z'},
 'status': 'DOCKED',
 'flightMode': 'BURN'}

In [11]:
def print_nav_info(ship):
    nav = F_nav.get_ship_nav(ship)
    if nav['status'] == 'IN_TRANSIT':
        arrival_time = nav['arrival']
        print(ship, "is navigating to", nav['waypointSymbol'], "- ETA", nav['arrival'], f"({F_nav.get_transit_time_seconds(ship):.0f} seconds remaining).")
    else:
        print(f"{ship} is {nav['status']} at {nav['waypointSymbol']}.")
        inv = F_trade.get_ship_cargo(ship)['inventory']
        if len(inv):
            print("[CARGO]")
            for i in inv:
                print(f"  - {i['units']} {i['symbol']}")

In [43]:
F_nav.refuel_ship('RYVIOS-1')
F_nav.navigate_in_system('RYVIOS-1', 'X1-TP30-A2', 'BURN')

[ERROR] Ship RYVIOS-1 not in system X1-TP30 (currently in X1-QT34). Navigation cancelled.


False

In [102]:
datetime.fromisoformat('2025-11-04T22:32:42.682+00:00')

datetime.datetime(2025, 11, 4, 22, 32, 42, 682000, tzinfo=datetime.timezone.utc)

In [37]:
constrsite = "X1-QT34-I60"
ST.get_request(f'/systems/X1-QT34/waypoints/{constrsite}/construction').json()

{'data': {'symbol': 'X1-QT34-I60',
  'materials': [{'tradeSymbol': 'FAB_MATS',
    'required': 1600,
    'fulfilled': 1600},
   {'tradeSymbol': 'ADVANCED_CIRCUITRY', 'required': 400, 'fulfilled': 400},
   {'tradeSymbol': 'QUANTUM_STABILIZERS', 'required': 1, 'fulfilled': 1}],
  'isComplete': True}}

In [31]:
F_nav.get_transit_time_seconds('RYVIOS-1')

-716.976149

In [21]:
F_trade.get_total_profit_from_trade('RYVIOS-1', 'X1-TP30-D41', 'X1-TP30-H52', '2025-11-07T09:27:00Z')

49365

In [14]:
fleet_resource_manager.get_available_ships_in_systems(["X1-QT34"], "SATELLITE", 1, "PROBE-MANAGER-X1-QT34")

['RYVIOS-2', 'RYVIOS-3']

In [10]:
i = 1.2345
print(f"num = {i:.0f}")

num = 1


In [44]:
ships = fleet_resource_manager.get_controller_fleet("PROBE-MANAGER-X1-QT34")
for s in ships:
    print_nav_info(s)

RYVIOS-2 is navigating to X1-QT34-J63 - ETA 2025-11-13T23:19:25.142Z (993 seconds remaining).
RYVIOS-3 is navigating to X1-QT34-F51 - ETA 2025-11-13T23:03:26.644Z (34 seconds remaining).


In [45]:
fleet_resource_manager.release_fleet("PROBE-MANAGER-X1-QT34", force=True)

True

In [17]:
fleet_resource_manager.release_ship("RYVIOS-4")

True

In [9]:
SIPHON.get_available_siphon_drones('X1-QT34', priority=100, controller="SIPHON-CONTROLLER-X1-QT34")

['RYVIOS-4']

In [ ]:
async def mine_goods(ship: str, waypoint : str, goods : list = None):
    """ Extracts from a waypoint until cargo hold is filled, keeping only the desired goods.
    """
    max_iterations = 50
    refresh_period = 10 # Time between checks if ship gets locked (full cargo)

    # Navigate to waypoint if not there
    if F_nav.get_ship_waypoint(ship) != waypoint:
        if not scripts.navigate(ship, waypoint):
            return False
        await scripts.await_navigation(ship)

    # Orbit location
    F_nav.orbit_ship(ship)

    # Continually extract from destination
    for _it in range(max_iterations):
        if F_extract.extract(ship):
            # Check if the good is desired; if not, jettison it immediately.
            # TODO: make this work with the cached cargo data (which is written on extraction)
            """
            if goods is not None:
                e_yield = data['extraction']['yield']
                if e_yield['symbol'] not in goods:
                    ST.post_request(f'/my/ships/{ship}/jettison', data={'symbol': e_yield['symbol'], 'units': e_yield['units']})
            """
            # Check cargo capacity
            cargo = F_trade.get_ship_cargo()
            if cargo['capacity'] <= cargo['units']:
            # Hold is full. Stop extracting and wait a while.
                await asyncio.sleep(refresh_period)
            else:
            # Otherwise, sleep until next extraction
                cd = F_utils.get_ship_cooldown(ship)['remainingSeconds']
                await asyncio.sleep(cd)
        else:
        # Extraction failed for some reason; idle for a while and then try again
            await asyncio.sleep(refresh_period)
            #print(f'[ERROR] Ship {ship} failed to extract. Aborting operation.')
            #print(f' [INFO]', r.json())

In [ ]:
### GLOBALS ###


fleet = dict()

# Try to acquire a full fleet
DESIRED_DRONES = 8 # Based on info from the Discord
if len(fleet) < DESIRED_DRONES:
    for d in SIPHON.get_available_siphon_drones('X1-QT34', priority=100, controller="SIPHON-CONTROLLER-X1-QT34"):
        

In [79]:
for i in range(1, 5):
    #F_trade._refresh_cargo('RYVIOS-'+str(i))
    print_nav_info("RYVIOS-"+str(i))

RYVIOS-1 is IN_ORBIT at X1-QT34-G54.
[CARGO]
  - 4 LIQUID_HYDROGEN
  - 19 HYDROCARBON
  - 11 LIQUID_NITROGEN
RYVIOS-2 is IN_ORBIT at X1-QT34-J63.
RYVIOS-3 is IN_ORBIT at X1-QT34-F51.
RYVIOS-4 is IN_ORBIT at X1-QT34-C39.
[CARGO]
  - 15 LIQUID_NITROGEN


In [13]:
F_utils.get_ship_cooldown("RYVIOS-4")

{'shipSymbol': 'RYVIOS-4',
 'totalSeconds': 0,
 'remainingSeconds': 0,
 'expiration': None}

In [70]:
F_trade._add_cargo('RYVIOS-1', {"symbol": 'SOME_BULLSHIT', "units": -2})

True

In [80]:
F_trade._refresh_cargo("RYVIOS-1")

True

In [56]:
F_trade.get_ship_cargo('RYVIOS-1')

{'capacity': 40,
 'units': 12,
 'inventory': [{'symbol': 'HYDROCARBON',
   'name': 'Hydrocarbons',
   'description': 'A highly-valued substance used in the production of fuels, plastics, and other industrial products.',
   'units': 15},
  {'symbol': 'LIQUID_HYDROGEN',
   'name': 'Liquid Hydrogen',
   'description': 'A highly efficient and lightweight fuel used in advanced propulsion systems.',
   'units': 4},
  {'symbol': 'SOME_BULLSHIT', 'name': None, 'description': None, 'units': 4}]}

In [72]:
F_nav._refresh_ship_nav("RYVIOS-1")

True

In [73]:
ST.get_request('/my/ships/RYVIOS-1').json()

{'data': {'symbol': 'RYVIOS-1',
  'registration': {'name': 'RYVIOS-1',
   'factionSymbol': 'COSMIC',
   'role': 'COMMAND'},
  'nav': {'systemSymbol': 'X1-QT34',
   'waypointSymbol': 'X1-QT34-C39',
   'route': {'destination': {'symbol': 'X1-QT34-C39',
     'type': 'GAS_GIANT',
     'systemSymbol': 'X1-QT34',
     'x': -154,
     'y': 10},
    'origin': {'symbol': 'X1-QT34-E46',
     'type': 'MOON',
     'systemSymbol': 'X1-QT34',
     'x': -41,
     'y': -38},
    'departureTime': '2025-11-14T12:59:58.323Z',
    'arrival': '2025-11-14T13:00:56.323Z'},
   'status': 'IN_ORBIT',
   'flightMode': 'BURN'},
  'crew': {'current': 57,
   'required': 57,
   'capacity': 80,
   'rotation': 'STRICT',
   'morale': 100,
   'wages': 0},
  'frame': {'symbol': 'FRAME_FRIGATE',
   'name': 'Frigate',
   'condition': 0.9912837718094404,
   'integrity': 1,
   'description': 'A medium-sized, multi-purpose spacecraft, often used for combat, transport, or support operations.',
   'moduleSlots': 8,
   'mounting

In [75]:
F_nav.get_fuel_required('X1-QT34-C39', 'X1-QT34-G54', 'BURN')

198

In [78]:
F_nav.refuel_ship('RYVIOS-1')
F_nav.navigate_in_system('RYVIOS-1', 'X1-QT34-G54', 'CRUISE')

[ERROR] Ship RYVIOS-1 failed to refuel.
 [INFO] {'error': {'code': 4601, 'message': 'Market purchase failed. Trade good FUEL is not available at X1-QT34-C39.', 'data': {'waypointSymbol': 'X1-QT34-C39', 'tradeSymbol': 'FUEL'}, 'requestId': '019a82ab-e55d-73ac-bf63-73b48ecb8933'}}
[INFO] Ship RYVIOS-1 en-route to X1-QT34-G54. Arrival at 2025-11-14T14:03:00.771 (85.0 seconds).


True